In [12]:
import cv2
import numpy as np
from PIL import Image


def draw_obb_on_image(img: Image.Image, obb_arr: np.ndarray) -> Image.Image:
    img_cv = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)

    for obb in obb_arr:
        cx, cy, w, h, angle_rad = obb
        angle_deg = np.degrees(angle_rad)

        # 중심 빨간색 점 (BGR 포맷)
        center = (int(cx), int(cy))
        cv2.circle(img_cv, center, radius=8, color=(0, 0, 255), thickness=10)  # 빨간 점

        # 회전된 바운딩 박스 4개 꼭짓점 계산
        rect = ((cx, cy), (w, h), angle_deg)
        box = cv2.boxPoints(rect)
        box = np.int32(box)
        cv2.polylines(img_cv, [box], isClosed=True, color=(255, 0, 0), thickness=10)  # 파란 사각형

    # 다시 RGB 포맷으로 변환하여 PIL 이미지로 반환
    img_out = cv2.cvtColor(img_cv, cv2.COLOR_BGR2RGB)
    return Image.fromarray(img_out)


# 사용 예
if __name__ == "__main__":
    img_path = r"C:\Users\zpipa\OneDrive\Desktop\yangjm\의공학교실\Surgical_Innovation\Nail_images\nail photo\psoriasis\20250804 33292933 박선민(3).JPG"
    img = Image.open(img_path).convert("RGB")

    obb_arr = np.array([[3959.9,      1218.4      ,521.24     , 446.88,      1.5383],
                        [2720.6,      579.85     , 454.96      , 520.9 ,   0.040828],
                        [       5323,      925.38    ,  548.45     ,  473.7  ,    1.5186],
                        [     4621.5,      704.13   ,   577.53    ,   480.1   ,   1.4643],
                        [     3336.2,      1303.7  ,    454.94   ,   495.69    , 0.10497],
                        [       2054,      850.34 ,     415.18  ,    538.85,     0.21761],
                        [       1258,     1775.5 ,     331.94  ,    436.64  ,   0.37715],
                        [     6276.6,      1819.5      ,560.44,      332.72  ,    1.1513]])

    img_with_obb = draw_obb_on_image(img, obb_arr)
    img_with_obb.show()  # 이미지 출력 또는 저장 가능


In [7]:
import cv2
import numpy as np
import math

def draw_obb(img, obb_arr):
    img_draw = img.copy()
    
    for obb in obb_arr:
        cx, cy, w, h, angle = obb

        # 중심점 (빨간 점)
        center = (int(cx), int(cy))
        cv2.circle(img_draw, center, 5, (0, 0, 255), thickness=-1)  # 붉은색 점

        # 각도를 라디안으로 변환 (angle은 y축부터 떨어진 각도)
        angle_deg = np.degrees(angle)
        
        # 회전 matrix 만들기
        rotation = cv2.getRotationMatrix2D(center, angle_deg, 1.0)
        
        # 사각형 좌표 (중심 기준, 회전 전)
        rect_pts = np.array([
            [cx - w / 2, cy - h / 2],
            [cx + w / 2, cy - h / 2],
            [cx + w / 2, cy + h / 2],
            [cx - w / 2, cy + h / 2]
        ], dtype=np.float32)
        
        # 회전 적용
        pts = cv2.transform(np.array([rect_pts]), rotation)[0]
        pts = pts.astype(int)

        # 바운딩 박스 그리기 (파란색 선 두께 2)
        cv2.polylines(img_draw, [pts], isClosed=True, color=(255, 0, 0), thickness=2)

        # 각도를 y축 기준 선 그리기 (검정색, 길이 100픽셀)
        length = 600
        angle_rad = -angle  # OpenCV coordinate system에서 y축 방향을 고려한 부호 반전        
        x_end = int(cx + length * math.sin(angle_rad))
        y_end = int(cy - length * math.cos(angle_rad))
        cv2.line(img_draw, center, (x_end, y_end), (0, 0, 0), 2)

    return img_draw

# 사용 예:
image_path = r"C:\Users\zpipa\OneDrive\Desktop\yangjm\의공학교실\Surgical_Innovation\Nail_images\nail photo\psoriasis\20250804 33292933 박선민(4).JPG"
img = cv2.imread(image_path)
obb_arr = np.array([[3161.5, 2021.1, 825.37, 819.99, 0.031721], [4078.8, 1963, 890.07, 838.59, 1.5598]])
img_out = draw_obb(img, obb_arr)
cv2.imshow("obb visualization", img_out)
cv2.waitKey(0)
cv2.destroyAllWindows()


AttributeError: 'NoneType' object has no attribute 'copy'